In [3]:
import numpy as np
import nibabel
import nilearn
from nilearn.image import resample_to_img
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.plotting import show
from nilearn.plotting import plot_roi
from nilearn import image
from nilearn.masking import apply_mask
# get_ipython().magic('matplotlib inline')
import scipy
import matplotlib
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.input_data import NiftiMasker
#from nilearn import plotting
import nibabel
from nilearn.masking import apply_mask
from nilearn.image import load_img
from nilearn.image import new_img_like
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, svm, metrics
from sklearn.linear_model import Ridge
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import KFold
from sklearn.cross_validation import LeaveOneLabelOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_selection import SelectFwe
from scipy import signal
from scipy.fftpack import fft, fftshift
from scipy import interp

params = {'legend.fontsize': 'large',
          'figure.figsize': (5, 3),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
font = {'weight': 'bold',
        'size': 22}
plt.rc('font', **font)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, f_classif, GenericUnivariateSelect, SelectKBest, chi2
from sklearn.feature_selection import RFE
import os
import seaborn as sns
import pandas as pd
import csv
from scipy import stats
import brainiak
import brainiak.funcalign.srm


In [4]:
# UPDATED PROJECT DIR & MASKS!!
projectDir='/jukebox/norman/amennen/prettymouth_fmriprep2/'
DMNmask='/jukebox/norman/amennen/MNI_things/Yeo_JNeurophysiol11_MNI152/Yeo_Network7mask_reoriented_resampledBOLD2.nii.gz'
fmriprep_dir=projectDir + '/derivatives/fmriprep'

In [5]:
# load subject numbers
subInd = 0
nsub=38
allnames = []
allgroups = []
groupInfo={}
# skip subjects 039 and 116
with open(projectDir + 'participants.tsv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        if 'sub' in row[0]:
            # now skip the subjects we don't want to analyze
            allInfo = row[0].split('\t')
            subjName=allInfo[0]
            if subjName != 'sub-039' and subjName != 'sub-116':
                if allInfo[3] == 'paranoia':
                    group = 0
                elif allInfo[3] == 'affair':
                    group = 1
                allnames.append(subjName)
                allgroups.append(group)
                subInd+=1

In [6]:
paranoidSubj = allnames[0:19]
cheatingSubj = allnames[19:]
paranoidLabel = allgroups[0:19]
cheatingLabel = allgroups[19:]
nfolds=19
print(allnames)

['sub-088', 'sub-089', 'sub-090', 'sub-091', 'sub-092', 'sub-093', 'sub-094', 'sub-095', 'sub-096', 'sub-097', 'sub-098', 'sub-099', 'sub-100', 'sub-101', 'sub-102', 'sub-103', 'sub-104', 'sub-105', 'sub-106', 'sub-107', 'sub-108', 'sub-109', 'sub-110', 'sub-111', 'sub-068', 'sub-081', 'sub-112', 'sub-053', 'sub-113', 'sub-031', 'sub-114', 'sub-115', 'sub-117', 'sub-118', 'sub-119', 'sub-120', 'sub-121', 'sub-122']


In [8]:
# make aggregate data matrix - USED --don't need to run again
nVox = 3757
nTRs = 475
nSub = 38
aggregate_masked_data = np.zeros((nVox,nTRs,nSub))
for s in np.arange(nsub):
   subjName=allnames[s]
   subjData=fmriprep_dir + '/' + subjName + '/' + 'func' + '/' + subjName + '_task-prettymouth_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
   masked_data = apply_mask(subjData, DMNmask)
   aggregate_masked_data[:,:,s] = masked_data.T

np.save('aggregate_data.npy', aggregate_masked_data)

In [9]:
images_concatenated = np.load('aggregate_data.npy')

In [12]:
images_concatenated.dtype # NOT integers--definitely floats!!

dtype('float64')

In [6]:
vox_num, nTR, num_subs = images_concatenated.shape  # Pull out the shape data

print('Participants ', num_subs)
print('Voxels per participant ', vox_num)
print('TRs per participant ', nTR)

Participants  38
Voxels per participant  3757
TRs per participant  475


In [27]:
np.shape(images_concatenated)

(3757, 475, 38)

In [8]:
train_data = []
test_data = []
for sub in range(num_subs):
    train_data.append(images_concatenated[:, :, sub])  # Take the first half of TRs as training (The double dash means integer division)
    test_data.append(images_concatenated[:, :, sub])  # Take the second half of TRs as testing (The double dash means integer division)

In [10]:
np.shape(train_data[0])

(3757, 475)

In [20]:
# Iterate through the subjects
for sub in range(num_subs):
    
    # Do it for training data
    train_data[sub] = stats.zscore(train_data[sub],
                                   axis=1,
                                   ddof=1)
    train_data[sub] = np.nan_to_num(train_data[sub])
    
    # Do it for test data
    test_data[sub] = stats.zscore(test_data[sub],
                                  axis=1,
                                  ddof=1)
    test_data[sub] = np.nan_to_num(test_data[sub])

/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/scipy/stats/stats.py:2246: RuntimeWarning: invalid value encountered in true_divide
  np.expand_dims(sstd, axis=axis))


In [24]:
features = 50  # How many features will you fit?
n_iter = 20  # How many iterations of fitting will you perform

# Create the SRM object
srm = brainiak.funcalign.srm.SRM(n_iter=n_iter, 
                                 features=features,
                                )

# Fit the SRM data
print('Fitting SRM, may take a few minutes ...')
srm.fit(train_data)

print('SRM has been fit')

Fitting SRM, may take a few minutes ...


KeyboardInterrupt: 

In [ ]:
# Plot the shared response
print('SRM: Features X Time Points ', srm.s_.shape)
plt.figure()
plt.title('SRM: Features X Time Points')
plt.imshow(srm.s_, cmap='viridis')
plt.colorbar()

In [ ]:
# Transform the test data into the shared space using the individual weight matrices
shared_test = srm.transform(test_data)

# Normalize data
for subject in range(num_subs):
    shared_test[subject] = stats.zscore(shared_test[subject], axis=1, ddof=1)

In [151]:
# try first way of converting everyone to a vector
accuracy = np.zeros(nfolds)
for f in np.arange(nfolds):
    testingSubjects = np.array([paranoidSubj[f],cheatingSubj[f]])
    testingInd = np.array([f,f+19])
    testingLabels = np.array([paranoidLabel[f],cheatingLabel[f]])
    indTrain = np.arange(19)
    trainingSubjectsInd=np.concatenate([indTrain[:f],indTrain[f+1:]])
    #trainingSubjects=[paranoidSubj[j] for j in trainingSubjectsInd]+[cheatingSubj[j] for j in trainingSubjectsInd]
    trainingLabels = [paranoidLabel[j] for j in trainingSubjectsInd]+[cheatingLabel[j] for j in trainingSubjectsInd]
    s1 = np.arange(19)
    s2 = np.arange(19) + 19
    trainingInd = [s1[j] for j in trainingSubjectsInd]+[s2[j] for j in trainingSubjectsInd]
    clf = LinearSVC()
    clf.fit(data_as_vector[trainingInd,:],trainingLabels)
    accuracy[f] = clf.score(data_as_vector[testingInd,:],testingLabels)
    print(f)
    print(accuracy[f])

0
0.5
1
0.5
2
0.5
3
1.0
4
1.0
5
0.5
6
1.0
7
1.0
8
0.5
9
1.0
10
0.5
11
1.0
12
0.5
13
0.5
14
1.0
15
0.5
16
1.0
17
0.5
18
1.0


In [ ]:
mean_acc = np.mean(accuracy)
print('mean accuracy is @2.2f' % mean_acc)
# first try: 73.6 % accuracy

In [142]:
trainingInd

[0,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37]

In [143]:
testingInd

array([ 1, 20])